In [1]:
import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))


from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet,COMPILER
from cimCommand.singleCmdInfo import *

In [2]:
chip=CHIP(PS("192.168.1.10", 7,debug=1),init=False)
# chip.set_device_cfg(0)
filename="../assembler_code/row_read_point3.txt"
chip.add_compiler("row_read_point3",filename=filename,encoding="utf-8")

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。


In [3]:
compiler = COMPILER()
compiler.load_assembler_ins(filename,encoding="utf-8")
ins_data = compiler.get_ins_data()
print(compiler)
chip.execute_ins(ins_data=ins_data,ins_ram_start=0)

const: count_max_c                   : 常量值: 512
const: pq_c                          : 常量值: 0
const: row_bank_din_ram_s_c          : 常量值: 0
const: row_bank_din_ram_e_c          : 常量值: 1
const: col_bank_din_ram_s_c          : 常量值: 2
const: col_bank_din_ram_e_c          : 常量值: 3
const: row_index_din_ram_s_c         : 常量值: 4
const: row_index_din_ram_e_c         : 常量值: 6
const: col_index_din_ram_s_c         : 常量值: 8
const: col_index_din_ram_e_c         : 常量值: 12

变量名: zero                          : 寄存器编号: 0
变量名: one                           : 寄存器编号: 1
变量名: four                          : 寄存器编号: 2
变量名: count                         : 寄存器编号: 3
变量名: count_max                     : 寄存器编号: 4
变量名: pq                            : 寄存器编号: 5
变量名: row_bank_addr                 : 寄存器编号: 6
变量名: row_bank_end_addr             : 寄存器编号: 7
变量名: row_index_start_addr          : 寄存器编号: 8
变量名: row_index_end_addr            : 寄存器编号: 9
变量名: row_bank_num                  : 寄存器编号: 10
变量名: row_bank_mask           

### 1.通过手动调用函数写汇编代码

In [4]:
compiler = COMPILER()
# 首先定义和声明能当立即数使用的常量变量
compiler.consti("row_bank_start_c",3)
compiler.consti("row_bank_end_c",4)
compiler.consti("col_bank_start_c",3)
compiler.consti("col_bank_end_c",4)

compiler.consti("count_max_c",512)
compiler.consti("pq_c",0)

compiler.consti("row_index_din_ram_s_c",0)
compiler.consti("row_index_din_ram_e_c",2)

compiler.consti("col_index_din_ram_s_c",4)
compiler.consti("col_index_din_ram_e_c",6)

# 正式的汇编代码开始
compiler.add_label("start")
compiler.addi("row_bank_start","zero","row_bank_start_c")
compiler.addi("row_bank_end","zero","row_bank_end_c")
compiler.addi("col_bank_start","zero","col_bank_start_c")
compiler.addi("col_bank_end","zero","col_bank_end_c")

compiler.addi("shift_one", "zero", 1)
compiler.addi("shift_four", "zero", 4)
compiler.addi("four", "zero", 4)
compiler.addi("count", "zero", 0)
compiler.addi("count_max", "zero", "count_max_c")
compiler.addi("pq", "zero", "pq_c")

# ------------------------------------------------------------
compiler.addi("row_index_start_addr","zero","row_index_din_ram_s_c")
compiler.addi("row_index_end_addr","zero","row_index_din_ram_e_c")

compiler.addi("row_bank_mask","zero",0x1)
compiler.sll("row_bank_mask","row_bank_mask","row_bank_start")
compiler.addi("row_bank_count","row_bank_start",0)

compiler.add_label("loop1")

# ------------------------------------------------------------
compiler.load_din_ram("row_index_count","row_index_start_addr")
compiler.load_din_ram("row_end_index","row_index_end_addr")
compiler.addi("row_index_mask", "zero", 0x1)
compiler.sll("row_index_mask", "row_index_mask", "row_index_count")
compiler.set_row_bank("row_bank_mask", "row_index_mask")

compiler.add_label("loop2")
compiler.addi("col_index_start_addr", "zero", "col_index_din_ram_s_c")
compiler.addi("col_index_end_addr", "zero", "col_index_din_ram_e_c")
compiler.addi("col_bank_mask", "zero", 0x1)
compiler.sll("col_bank_mask","col_bank_mask","col_bank_start")
compiler.addi("col_bank_count", "col_bank_start", 0)

compiler.add_label("loop3")
compiler.load_din_ram("col_index_count","col_index_start_addr")
compiler.load_din_ram("col_end_index","col_index_end_addr")
compiler.addi("col_index_mask", "zero", 0x1)
compiler.sll("col_index_mask","col_index_mask","col_index_count")

compiler.bge("col_bank_count", "four", "tia_big4")
compiler.add_label("tia_nobig4")
compiler.add("tmp_bank", "col_bank_count", "four")
compiler.jump("tia_end")
compiler.add_label("tia_big4")
compiler.sub("tmp_bank", "col_bank_count", "four")
compiler.add_label("tia_end")
compiler.sll("tmp_bank", "tmp_bank", "shift_one")

compiler.add_label("loop4")
compiler.srl("tmp_index", "col_index_count", "shift_four")
compiler.add("tia", "tmp_bank", "tmp_index")
compiler.set_col_bank("col_bank_mask", "col_index_mask")

compiler.read_row_pulse_tia("tia", "count", "pq")
compiler.addi("count", "count", 1)
compiler.bge("count", "count_max", "return_res")
compiler.jump("jump_return")
compiler.add_label("return_res")
compiler.return_dout("count", "zero", "pq")
compiler.xori("pq", "pq", 1)
compiler.addi("count", "zero", 0)
compiler.add_label("jump_return")
compiler.bge("col_index_count", "col_end_index", "end4")
compiler.addi("col_index_count", "col_index_count", 1)
compiler.sll("col_index_mask", "col_index_mask", "shift_one")
compiler.jump("loop4")

compiler.add_label("end4")
compiler.set_col_bank("col_bank_mask", "zero")
compiler.bge("col_bank_count", "col_bank_end", "end3")
compiler.addi("col_bank_count", "col_bank_count", 1)
compiler.sll("col_bank_mask", "col_bank_mask", "shift_one")
compiler.addi("col_index_start_addr", "col_index_start_addr", 1)
compiler.addi("col_index_end_addr", "col_index_end_addr", 1)
compiler.jump("loop3")

compiler.add_label("end3")
compiler.bge("row_index_count", "row_end_index", "end2")
compiler.addi("row_index_count", "row_index_count", 1)
compiler.sll("row_index_mask", "row_index_mask", "shift_one")
compiler.jump("loop2")

compiler.add_label("end2")
compiler.set_row_bank("row_bank_mask", "zero")
compiler.bge("row_bank_count", "row_bank_end", "end1")
compiler.addi("row_bank_count", "row_bank_count", 1)
compiler.sll("row_bank_mask", "row_bank_mask", "shift_one")
compiler.addi("row_index_start_addr", "row_index_start_addr", 1)
compiler.addi("row_index_end_addr", "row_index_end_addr", 1)
compiler.jump("loop1")

compiler.add_label("end1")
compiler.return_dout("count", "zero", "pq")
compiler.exit()

pkt = Packet()
pkt.append_single(compiler.get_ins_data(),mode=4)
ins1= str(pkt)

### 2.从汇编文件中加载汇编代码

In [3]:
compiler = COMPILER()
compiler.load_assembler_ins("../assembler_code/汇编测试代码.txt",encoding="utf-8")
print(compiler)
# compiler.add_offset(offset=10)
# print(compiler)
# compiler.add_offset(offset=9)
# pkt = Packet()
# pkt.append_single(compiler.get_ins_data(),mode=4)
# ins2= str(pkt)

# print(compiler)

const: imm                           : 常量值: 1
const: imm0                          : 常量值: 3
const: imm1                          : 常量值: 7

变量名: zero                          : 寄存器编号: 0
变量名: reg0                          : 寄存器编号: 1
变量名: reg1                          : 寄存器编号: 2
变量名: reg2                          : 寄存器编号: 3

标签: label                         : 指令地址: 3

0		consti	imm, 1
0		consti	imm0, 3
0		consti	imm1, 7
0		addi	reg0, zero, 0x1
1		addi	reg1, zero, 0x3
2		addi	reg2, zero, 0x7

3	label:
3		addi	reg1, reg0, imm
4		add	reg2, reg1, reg0
5		sub	reg2, reg1, reg0
6		xori	reg1, reg0, imm
7		sll	reg2, reg1, reg0
8		srl	reg2, reg1, reg0
9		bge	reg1, reg0, label
10		jump	label
11		exit	
12		load_din_ram	reg1, reg0
13		set_row_bank	reg1, reg0
14		set_col_bank	reg1, reg0
15		read_row_pulse_tia	reg2, reg1, reg0
16		read_col_pulse_tia	reg2, reg1, reg0
17		return_dout	reg2, reg1, reg0
18		dac_v	imm1, imm0
19		row_bank	imm1, imm0
20		col_bank	imm1, imm0
21		read_row_pulse	imm
22		read_col_

### 3.调试相关函数

In [7]:
# 显示汇编指令
print(compiler.get_assembler_ins())

	consti	count_max_c, 512
	consti	pq_c, 0
	consti	row_bank_din_ram_s_c, 0
	consti	row_bank_din_ram_e_c, 2
	consti	col_bank_din_ram_s_c, 2
	consti	col_bank_din_ram_e_c, 4
	consti	row_index_din_ram_s_c, 4
	consti	row_index_din_ram_e_c, 6
	consti	col_index_din_ram_s_c, 8
	consti	col_index_din_ram_e_c, 12

start:	
	addi	one, zero, 1
	addi	four, zero, 4
	addi	count, zero, 0
	addi	count_max, zero, count_max_c
	addi	pq, zero, pq_c
	addi	row_bank_addr, zero, row_bank_din_ram_s_c
	addi	row_bank_end_addr, zero, row_bank_din_ram_e_c
	addi	row_index_start_addr, zero, row_index_din_ram_s_c
	addi	row_index_end_addr, zero, row_index_din_ram_e_c

loop1:	
	load_din_ram	row_bank_num, row_bank_addr
	sll	row_bank_mask, one, row_bank_num
	load_din_ram	row_index_num, row_index_start_addr
	load_din_ram	row_end_index, row_index_end_addr

loop2:	
	sll	row_index_mask, one, row_index_num
	set_row_bank	row_bank_mask, row_index_mask
	addi	col_bank_addr, zero, col_bank_din_ram_s_c
	addi	col_bank_end_addr, zero, col_

In [21]:
# 显示编译时的相关参数, 用于调试
print(compiler)

const: row_bank_start_c              : 常量值: 3
const: row_bank_end_c                : 常量值: 4
const: col_bank_start_c              : 常量值: 3
const: col_bank_end_c                : 常量值: 4
const: count_max_c                   : 常量值: 512
const: pq_c                          : 常量值: 0
const: row_index_din_ram_s_c         : 常量值: 0
const: row_index_din_ram_e_c         : 常量值: 2
const: col_index_din_ram_s_c         : 常量值: 4
const: col_index_din_ram_e_c         : 常量值: 6

变量名: zero                          : 寄存器编号: 0
变量名: row_bank_start                : 寄存器编号: 1
变量名: row_bank_end                  : 寄存器编号: 2
变量名: col_bank_start                : 寄存器编号: 3
变量名: col_bank_end                  : 寄存器编号: 4
变量名: shift_one                     : 寄存器编号: 5
变量名: shift_four                    : 寄存器编号: 6
变量名: four                          : 寄存器编号: 7
变量名: count                         : 寄存器编号: 8
变量名: count_max                     : 寄存器编号: 9
变量名: pq                            : 寄存器编号: 10
变量名: row_index_start_addr     

In [22]:
# 显示指令对应的字节码
pkt = Packet()
pkt.append_single(compiler.get_ins_data(),mode=4)
print(pkt)

模式: 4
	帧头:                      	字节码: 55 aa 04
	指令: pl_addi              	字节码: 0a 03 01 00
	指令: pl_addi              	字节码: 0a 04 02 00
	指令: pl_addi              	字节码: 0a 03 03 00
	指令: pl_addi              	字节码: 0a 04 04 00
	指令: pl_addi              	字节码: 0a 01 05 00
	指令: pl_addi              	字节码: 0a 04 06 00
	指令: pl_addi              	字节码: 0a 04 07 00
	指令: pl_addi              	字节码: 0a 00 08 00
	指令: pl_addi              	字节码: 0a 00 09 00
	指令: pl_addi              	字节码: 0a 00 0a 00
	指令: pl_addi              	字节码: 0a 00 0b 00
	指令: pl_addi              	字节码: 0a 02 0c 00
	指令: pl_addi              	字节码: 0a 01 0d 00
	指令: pl_sll               	字节码: 10 0d 0d 01
	指令: pl_addi              	字节码: 0a 00 0e 01
	指令: pl_load_din_ram      	字节码: 0c 00 0f 0b
	指令: pl_load_din_ram      	字节码: 0c 00 10 0c
	指令: pl_addi              	字节码: 0a 01 11 00
	指令: pl_sll               	字节码: 10 11 11 0f
	指令: pl_set_row_bank      	字节码: 12 00 0d 11
	指令: pl_addi              	字节码: 0a 04 12 00
	指令: pl_addi              	字节

In [23]:
ins1 = ins1.split("\n")
ins2 = ins2.split("\n")
for i in range(len(ins1)):
    if ins1[i]!= ins2[i]:
        print(i,"   ",ins1[i],ins2[i])
print(ins1==ins2)

True
